In [1]:
df = spark.createDataFrame([
        (1, 144.5, 5.9, 33, 'M'),
        (2, 167.2, 5.4, 45, 'M'),
        (3, 124.1, 5.2, 23, 'F'),
        (4, 144.5, 5.9, 33, 'M'),
        (5, 133.2, 5.7, 54, 'F'),
        (3, 124.1, 5.2, 23, 'F'),
        (5, 129.2, 5.3, 42, 'M'),
    ], ['id', 'weight', 'height', 'age', 'gender'])

In [2]:
display(df)

In [3]:
df.select('age').distinct().collect()

In [4]:
df.distinct().count()

In [5]:
#this will remove if entire row is same including id
df = df.dropDuplicates()

In [6]:
display(df)

In [7]:
df.select(['weight','height','age','gender']).distinct().collect()

In [8]:
new_df = df.select([c for c in df.columns if c!= 'id']).distinct()

In [10]:
import pyspark.sql.functions as fn

display(df)

In [11]:
df.agg( fn.count('id').alias('count'), fn.countDistinct('gender').alias('distinct')).show()

In [12]:
df.withColumn('new_id', fn.monotonically_increasing_id()).show()

In [13]:
df_miss = spark.createDataFrame([
        (1, 143.5, 5.6, 28,   'M',  100000),
        (2, 167.2, 5.4, 45,   'M',  None),
        (3, None , 5.2, None, None, None),
        (4, 144.5, 5.9, 33,   'M',  None),
        (5, 133.2, 5.7, 54,   'F',  None),
        (6, 124.1, 5.2, None, 'F',  None),
        (7, 129.2, 5.3, 42,   'M',  76000),
    ], ['id', 'weight', 'height', 'age', 'gender', 'income'])

In [14]:
df_miss.rdd.map(lambda row: (row['id'], sum([ c == None for c in row ]))).collect()

In [15]:
df_miss.where('id == 3').show()

In [16]:
#Now, find out what is the missing data percentage
# The * argument to the count() method instruct the method to count all row
# fn.count('age') - returns count of non-None elements
# * before the list - this that .agg() method to treat the list as a set of different paramerts
df_miss.agg( * [ ( 1 - (fn.count(c)/ fn.count('*'))).alias(c+'_missing')   for c in df_miss.columns] ).show()

In [17]:
def f(name,age):
  print name,age

f(*['abc',33])

In [18]:
# remove rows which have more than 3 entries null
df_miss.dropna(thresh=3).show()

In [19]:
df_miss.select([c for c in df_miss.columns if c != 'income']).show()

In [20]:
# Find outliers - observations which deviate from the distribution of rest of the sample
df_outliers = spark.createDataFrame([
        (1, 143.5, 5.3, 28),
        (2, 154.2, 5.5, 45),
        (3, 342.3, 5.1, 99),
        (4, 144.5, 5.5, 33),
        (5, 133.2, 5.4, 54),
        (6, 124.1, 5.1, 21),
        (7, 129.2, 5.3, 42),
    ], ['id', 'weight', 'height', 'age'])

In [21]:
cols = ['weight', 'height', 'age']
bounds = {}

for col in cols:
  qualtiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
  
  IQR = qualtiles[1] - qualtiles[0]
  
  bounds[col] = [
    qualtiles[0] - IQR * 1.5,
    qualtiles[1] - IQR * 1.5
  ]

In [22]:
bounds

In [23]:
qualtiles